In [1]:
# Install required packages
!pip install transformers flask flask-cors pyngrok --quiet
!pip install sentence-transformers scikit-learn --quiet
!pip install sacremoses --quiet

# Load Models

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT-Large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

biobert = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb', device=device)

# Document Embeddings & Retriver

In [5]:
docs = []
doc_embeddings = []

In [6]:
def add_doc(text):
    # Embed the incoming doc using BioBERT
    embedding = biobert.encode([text], convert_to_tensor=True)[0].cpu().numpy()

    # Store in memory
    docs.append(text)
    doc_embeddings.append(embedding)

    print(f"✅ Added document. Total docs: {len(docs)}")

In [7]:
def find_most_similar_doc(query, top_k=1):
    if not doc_embeddings:
        return "❌ No documents in memory."

    # Embed the user query
    query_embedding = biobert.encode([query], convert_to_tensor=True)[0].cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # Get the top-k most similar docs
    top_indices = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "doc": docs[idx],
            "similarity": float(similarities[idx])
        })

    return results

# API

In [8]:
!pip install flask flask-cors

In [9]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/embed-doc', methods=['POST'])
def embed_doc():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "Missing 'text' field."}), 400

    add_doc(text)
    return jsonify({"message": "✅ Document embedded.", "total_docs": len(docs)})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field."}), 400

    results = find_most_similar_doc(query, top_k=1)
    return jsonify(results)

In [10]:
!pip install pyngrok

In [ ]:
# ⚠️ CRITICAL: Make sure you ran cells 2-8 BEFORE running this cell!

# Check if required variables exist
required_vars = ['tokenizer', 'model', 'biobert', 'docs', 'doc_embeddings', 'add_doc', 'find_most_similar_doc']
missing_vars = []
for var in required_vars:
    if var not in globals():
        missing_vars.append(var)

if missing_vars:
    print("❌ ERROR: Missing required variables:", missing_vars)
    print("🔄 SOLUTION: Please run cells 2-8 in order first!")
    raise Exception(f"Missing variables: {missing_vars}. Run cells 2-8 first!")

print("✅ All required variables found! Starting Flask app...")

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import torch

app = Flask(__name__)
CORS(app)

# Add some sample health documents for testing
sample_docs = [
    "Blood pressure is typically measured in millimeters of mercury (mmHg). Normal blood pressure for adults is usually around 120/80 mmHg. High blood pressure (hypertension) is defined as 140/90 mmHg or higher.",
    "Diabetes is a chronic condition that affects how your body processes blood sugar (glucose). Type 1 diabetes occurs when the pancreas produces little or no insulin. Type 2 diabetes occurs when your body becomes resistant to insulin.",
    "Regular exercise is essential for maintaining good health. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, plus muscle-strengthening activities twice a week.",
    "Sleep is crucial for physical and mental health. Adults typically need 7-9 hours of sleep per night. Poor sleep can affect immune function, mood, and cognitive performance."
]

# Add sample documents to our embedding system
for doc in sample_docs:
    add_doc(doc)

def generate_biogpt_response(query, context="", max_tokens=200):
    """Generate response using BioGPT model"""
    try:
        # Create prompt with context
        prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
        
        # Tokenize and generate
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + max_tokens,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decode response
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the answer part
        answer = generated_text.split("Answer:")[-1].strip()
        
        return answer
        
    except Exception as e:
        print(f"Error generating response: {e}")
        return f"I apologize, but I encountered an error while processing your question about {query}. Please try rephrasing your question."

@app.route('/')
def index():
    return "WellnessGrid AI Flask API is running!"

@app.route('/ask', methods=['POST'])
def ask():
    print("📥 Received /ask request")
    try:
        data = request.json
        query = data.get("query", "") or data.get("question", "")
        user_context = data.get("userContext", {})
        
        print(f"Processing query: {query}")
        print(f"User context: {user_context}")
        
        if not query:
            return jsonify({
                "response": "Please provide a question.",
                "sources": [],
                "mockMode": False
            }), 400
        
        # Find relevant documents using BioBERT
        similar_docs = find_most_similar_doc(query, top_k=3)
        
        # Prepare context from similar documents
        context = ""
        sources = []
        
        if isinstance(similar_docs, list) and similar_docs:
            for i, doc_info in enumerate(similar_docs):
                if isinstance(doc_info, dict):
                    context += f"Document {i+1}: {doc_info['doc']}\n\n"
                    sources.append({
                        "title": f"Health Document {i+1}",
                        "content": doc_info['doc'][:200] + "...",
                        "similarity": f"{doc_info['similarity']:.3f}"
                    })
        
        # Add user health conditions to context if available
        if user_context.get("healthConditions"):
            conditions = ", ".join(user_context["healthConditions"])
            context = f"Patient has the following health conditions: {conditions}\n\n{context}"
        
        # Generate AI response using BioGPT
        ai_response = generate_biogpt_response(query, context, max_tokens=150)
        
        # Format response for the app
        response = {
            "response": ai_response,
            "sources": sources,
            "mockMode": False
        }
        
        print(f"Generated response: {ai_response[:100]}...")
        print(f"Sources count: {len(sources)}")
        
        return jsonify(response)
        
    except Exception as e:
        print(f"Error in /ask endpoint: {e}")
        return jsonify({
            "response": "I apologize, but I'm experiencing technical difficulties. Please try again later.",
            "sources": [],
            "mockMode": True,
            "error": str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        "status": "healthy",
        "models": {
            "biogpt": "loaded",
            "biobert": "loaded"
        },
        "documents": len(docs)
    })

# Start Flask app in a thread
def run_app():
    app.run(port=5000, debug=False)

# Start the Flask app
thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

# Setup ngrok
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)
print("🤖 AI Models loaded and ready!")
print("📚 Sample documents embedded:", len(docs))


In [ ]:
token = input('Enter ngrok auth token: ')

In [12]:
from pyngrok import ngrok

# Replace with your token from ngrok.com
ngrok.set_auth_token(token)

In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import torch

app = Flask(__name__)
CORS(app)

# Add some sample health documents for testing
sample_docs = [
    "Blood pressure is typically measured in millimeters of mercury (mmHg). Normal blood pressure for adults is usually around 120/80 mmHg. High blood pressure (hypertension) is defined as 140/90 mmHg or higher.",
    "Diabetes is a chronic condition that affects how your body processes blood sugar (glucose). Type 1 diabetes occurs when the pancreas produces little or no insulin. Type 2 diabetes occurs when your body becomes resistant to insulin.",
    "Regular exercise is essential for maintaining good health. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, plus muscle-strengthening activities twice a week.",
    "Sleep is crucial for physical and mental health. Adults typically need 7-9 hours of sleep per night. Poor sleep can affect immune function, mood, and cognitive performance."
]

# Add sample documents to our embedding system
for doc in sample_docs:
    add_doc(doc)

def generate_biogpt_response(query, context="", max_tokens=200):
    """Generate response using BioGPT model"""
    try:
        # Create prompt with context
        prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
        
        # Tokenize and generate
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + max_tokens,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decode response
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the answer part
        answer = generated_text.split("Answer:")[-1].strip()
        
        return answer
        
    except Exception as e:
        print(f"Error generating response: {e}")
        return f"I apologize, but I encountered an error while processing your question about {query}. Please try rephrasing your question."

@app.route('/')
def index():
    return "WellnessGrid AI Flask API is running!"

@app.route('/ask', methods=['POST'])
def ask():
    print("📥 Received /ask request")
    try:
        data = request.json
        query = data.get("query", "") or data.get("question", "")
        user_context = data.get("userContext", {})
        
        print(f"Processing query: {query}")
        print(f"User context: {user_context}")
        
        if not query:
            return jsonify({
                "response": "Please provide a question.",
                "sources": [],
                "mockMode": false
            }), 400
        
        # Find relevant documents using BioBERT
        similar_docs = find_most_similar_doc(query, top_k=3)
        
        # Prepare context from similar documents
        context = ""
        sources = []
        
        if isinstance(similar_docs, list) and similar_docs:
            for i, doc_info in enumerate(similar_docs):
                if isinstance(doc_info, dict):
                    context += f"Document {i+1}: {doc_info['doc']}\n\n"
                    sources.append({
                        "title": f"Health Document {i+1}",
                        "content": doc_info['doc'][:200] + "...",
                        "similarity": f"{doc_info['similarity']:.3f}"
                    })
        
        # Add user health conditions to context if available
        if user_context.get("healthConditions"):
            conditions = ", ".join(user_context["healthConditions"])
            context = f"Patient has the following health conditions: {conditions}\n\n{context}"
        
        # Generate AI response using BioGPT
        ai_response = generate_biogpt_response(query, context, max_tokens=150)
        
        # Format response for the app
        response = {
            "response": ai_response,
            "sources": sources,
            "mockMode": false
        }
        
        print(f"Generated response: {ai_response[:100]}...")
        print(f"Sources count: {len(sources)}")
        
        return jsonify(response)
        
    except Exception as e:
        print(f"Error in /ask endpoint: {e}")
        return jsonify({
            "response": "I apologize, but I'm experiencing technical difficulties. Please try again later.",
            "sources": [],
            "mockMode": true,
            "error": str(e)
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        "status": "healthy",
        "models": {
            "biogpt": "loaded",
            "biobert": "loaded"
        },
        "documents": len(docs)
    })

# Start Flask app in a thread
def run_app():
    app.run(port=5000, debug=False)

# Start the Flask app
thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

# Setup ngrok
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)
print("🤖 AI Models loaded and ready!")
print("📚 Sample documents embedded:", len(docs))

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


🌐 Public URL: NgrokTunnel: "https://3afd-34-87-65-143.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
import os
import signal
from pyngrok import ngrok

# Kill ngrok
ngrok.kill()

# Kill the Flask thread by killing the whole Colab process (if needed)
os.kill(os.getpid(), signal.SIGKILL)

print('Killed')